# 04. Advanced Models - Deep Learning & Explainability (V2.0)

This notebook demonstrates the V2.0 models:
1. **GRU4Rec** - Session-based recommendations with RNN
2. **SASRec** - Self-Attentive Sequential Recommendation
3. **Two-Tower** - Dual encoder with FAISS retrieval
4. **Explainable AI** - Human-readable recommendation explanations

## Scientific Contributions
- **Novelty #2**: Session + History Fusion with attention-based combination
- **Novelty #3**: Explainable e-commerce recommendations with funnel awareness

In [ ]:
import sys
sys.path.insert(0, '..')

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from loguru import logger

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print('Setup complete!')

## 1. Load and Prepare Data

In [ ]:
from src.data.loaders.retailrocket import RetailRocketLoader
from src.data.processors.session_builder import SessionBuilder
from src.data.processors.splitter import TimeBasedSplitter
from src.data.features.user_features import UserFeatureExtractor
from src.config import settings

# Load data
loader = RetailRocketLoader()
events = loader.load_events()

print(f"Total events: {len(events):,}")
print(f"Unique users: {events['visitor_id'].nunique():,}")
print(f"Unique items: {events['item_id'].nunique():,}")

In [ ]:
# Build sessions
session_builder = SessionBuilder(timeout_minutes=30)
events = session_builder.build_sessions(events)

# Filter short sessions for sequential models
events_filtered = session_builder.filter_short_sessions(events, min_length=2)

print(f"\nFiltered events: {len(events_filtered):,}")
print(f"Sessions: {events_filtered['session_id'].nunique():,}")

In [ ]:
# Split data
splitter = TimeBasedSplitter(
    train_ratio=settings.train_ratio,
    val_ratio=settings.val_ratio,
    test_ratio=settings.test_ratio
)
train, val, test = splitter.split(events_filtered)

print(f"Train: {len(train):,}")
print(f"Val: {len(val):,}")
print(f"Test: {len(test):,}")

In [ ]:
# Extract user features
user_feature_extractor = UserFeatureExtractor()
user_feature_extractor.fit(train)

## 2. GRU4Rec - Session-based RNN

In [ ]:
from src.models.sequential.gru4rec import GRU4RecRecommender
from src.evaluation.evaluator import Evaluator

# Initialize evaluator
evaluator = Evaluator(k_values=[5, 10, 20])

In [ ]:
# Train GRU4Rec
gru4rec = GRU4RecRecommender(
    embedding_dim=64,
    hidden_dim=128,
    n_layers=1,
    dropout=0.2,
    learning_rate=0.001,
    batch_size=512,
    epochs=5,  # Reduce for demo
    loss_type='ce'
)

gru4rec.fit(train)

In [ ]:
# Evaluate GRU4Rec
gru4rec_results = evaluator.evaluate(
    gru4rec, train, test,
    n_items=20, max_users=2000
)

print("GRU4Rec Results:")
for k, v in gru4rec_results.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.4f}")

In [ ]:
# Demo: Session-based recommendation
sample_session = train.groupby('session_id').apply(
    lambda x: x['item_id'].tolist()
).iloc[0][:5]

print(f"Sample session items: {sample_session}")
recs = gru4rec.recommend_by_item_ids(sample_session, n_items=5)
print(f"\nRecommendations:")
for item_id, score in recs:
    print(f"  Item {item_id}: {score:.4f}")

## 3. SASRec - Self-Attentive Sequential

In [ ]:
from src.models.sequential.sasrec import SASRecRecommender

# Train SASRec
sasrec = SASRecRecommender(
    hidden_dim=64,
    n_heads=2,
    n_layers=2,
    max_seq_length=50,
    dropout=0.2,
    learning_rate=0.001,
    batch_size=512,
    epochs=5  # Reduce for demo
)

sasrec.fit(train)

In [ ]:
# Evaluate SASRec
sasrec_results = evaluator.evaluate(
    sasrec, train, test,
    n_items=20, max_users=2000
)

print("SASRec Results:")
for k, v in sasrec_results.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.4f}")

## 4. Two-Tower with FAISS

In [ ]:
from src.models.retrieval.two_tower import TwoTowerRecommender

# Train Two-Tower
two_tower = TwoTowerRecommender(
    embedding_dim=64,
    hidden_dims=[128, 64],
    dropout=0.2,
    temperature=0.1,
    learning_rate=0.001,
    batch_size=1024,
    epochs=5,  # Reduce for demo
    negative_samples=4,
    use_faiss=True
)

two_tower.fit(train)

In [ ]:
# Evaluate Two-Tower
two_tower_results = evaluator.evaluate(
    two_tower, train, test,
    n_items=20, max_users=2000
)

print("Two-Tower Results:")
for k, v in two_tower_results.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.4f}")

In [ ]:
# Demo: Similar items using Two-Tower embeddings
sample_item = train['item_id'].value_counts().index[0]
similar_items = two_tower.get_similar_items(sample_item, n_items=5)

print(f"Items similar to {sample_item}:")
for item_id, score in similar_items:
    print(f"  Item {item_id}: {score:.4f}")

## 5. Explainable AI

In [ ]:
from src.models.explainable.explainer import RecommendationExplainer

# Get item popularity for explainer
item_popularity = train['item_id'].value_counts().to_dict()

# Initialize explainer
explainer = RecommendationExplainer(
    user_feature_extractor=user_feature_extractor,
    item_popularity=item_popularity
)
explainer.fit(train)

In [ ]:
# Demo: Generate explanations
sample_user = train['visitor_id'].iloc[0]
recs = sasrec.recommend(sample_user, n_items=5)

print(f"Recommendations for user {sample_user}:")
print("=" * 60)

explanations = explainer.explain_batch(
    user_id=sample_user,
    recommendations=recs,
    model_name='sasrec'
)

for exp in explanations:
    print(f"\nItem {exp.item_id} (score: {exp.score:.4f})")
    print(f"Summary: {exp.get_summary()}")
    print(f"Confidence: {exp.confidence:.2f}")
    if len(exp.reasons) > 1:
        print("All reasons:")
        print(exp.get_full_explanation())

## 6. Session + History Fusion Hybrid (Novelty #2)

In [ ]:
from src.models.hybrid.funnel_aware import FunnelAwareHybridRecommender
from src.models.baselines.popular import PopularItemsRecommender
from src.models.collaborative.als import ALSRecommender
from src.models.content.item2vec import Item2VecRecommender

# Train component models
popular = PopularItemsRecommender()
popular.fit(train)

als = ALSRecommender(factors=64, iterations=10)
als.fit(train)

item2vec = Item2VecRecommender(embedding_dim=64, window=5, epochs=5)
item2vec.fit(train)

In [ ]:
# Create hybrid with session model
hybrid = FunnelAwareHybridRecommender(
    popular_model=popular,
    content_model=item2vec,
    cf_model=als,
    session_model=sasrec,  # Use SASRec for session component
    score_normalization='minmax',
    user_feature_extractor=user_feature_extractor
)
hybrid.fit(train)

In [ ]:
# Evaluate hybrid
hybrid_results = evaluator.evaluate(
    hybrid, train, test,
    n_items=20, max_users=2000
)

print("Session+History Fusion Hybrid Results:")
for k, v in hybrid_results.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.4f}")

## 7. Results Comparison

In [ ]:
# Collect all results
all_results = [
    gru4rec_results,
    sasrec_results,
    two_tower_results,
    hybrid_results
]

results_df = pd.DataFrame(all_results)
results_df = results_df.sort_values('ndcg@10', ascending=False)

# Display results
display_cols = ['model', 'precision@10', 'recall@10', 'ndcg@10', 'hit_rate', 'mrr']
display_cols = [c for c in display_cols if c in results_df.columns]
results_df[display_cols].round(4)

In [ ]:
# Plot comparison
fig, ax = plt.subplots(figsize=(10, 6))

metrics = ['precision@10', 'recall@10', 'ndcg@10']
models = results_df['model'].tolist()

x = np.arange(len(models))
width = 0.25

for i, metric in enumerate(metrics):
    values = results_df[metric].tolist()
    ax.bar(x + i * width, values, width, label=metric)

ax.set_xlabel('Model')
ax.set_ylabel('Score')
ax.set_title('V2.0 Model Comparison')
ax.set_xticks(x + width)
ax.set_xticklabels(models, rotation=45, ha='right')
ax.legend()

plt.tight_layout()
plt.show()

## 8. Visualize Item Embeddings

In [ ]:
from sklearn.manifold import TSNE

# Get embeddings from Two-Tower
item_embeddings = two_tower.get_item_embeddings()

# Sample for visualization
n_sample = min(5000, len(item_embeddings))
sample_idx = np.random.choice(len(item_embeddings), n_sample, replace=False)
sample_embeddings = item_embeddings[sample_idx]

# Apply t-SNE
print("Applying t-SNE...")
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
embeddings_2d = tsne.fit_transform(sample_embeddings)

In [ ]:
# Plot t-SNE
fig, ax = plt.subplots(figsize=(12, 10))

scatter = ax.scatter(
    embeddings_2d[:, 0],
    embeddings_2d[:, 1],
    alpha=0.5,
    s=5
)

ax.set_title('Item Embeddings (Two-Tower) - t-SNE Visualization')
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')

plt.tight_layout()
plt.show()

## 9. Summary

### Key Findings

1. **Sequential models** (GRU4Rec, SASRec) capture temporal patterns in user behavior
2. **Two-Tower** enables efficient retrieval with FAISS at scale
3. **Session + History Fusion** combines short-term (session) and long-term (history) signals
4. **Explainable AI** provides human-readable explanations with funnel awareness

### Scientific Contributions

- **Novelty #2**: Attention-based fusion of session and history signals
- **Novelty #3**: Multi-strategy explainability (collaborative, content, intent, session)

In [ ]:
# Save results
results_df.to_csv('../data/processed/v2_notebook_results.csv', index=False)
print("Results saved!")